## general helpers

In [3]:
CRS_4326 = "EPSG:4326"
CRS_3857 = "EPSG:3857"
CRS_2056 = "EPSG:2056"


from contextlib import contextmanager
import os

@contextmanager
def cd(newdir):
    prevdir = os.getcwd()
    os.chdir(os.path.expanduser(newdir))
    try:
        yield
    finally:
        os.chdir(prevdir)

## conversion helpers

In [4]:
import rasterio
import pyproj

from shapely.geometry import box
from shapely.ops import transform

def crs_convert_shapely(shape, in_crs=CRS_2056, out_crs=CRS_2056):
    if in_crs == out_crs:
        return shape

    in_crs = pyproj.CRS(in_crs)
    out_crs = pyproj.CRS(out_crs)
    projection = pyproj.Transformer.from_crs(in_crs, out_crs, always_xy=True).transform
    return transform(projection, shape)


def crs_convert_geometries(shapes:list, in_crs=CRS_2056, out_crs=CRS_2056):
    df = geopandas.GeoSeries(shapes)
    df = df.set_crs(in_crs)
    df = df.geometry.to_crs(out_crs)
    return df.values


def get_bbox_polygon_for_image(image_src, input_crs=CRS_2056, out_crs=CRS_2056):
    with rasterio.open(image_src) as src:
        # bounds are always reported in wgs84 coordinates
        bounds = src.bounds
        epsg_def = str(src.crs)
    # shapely.geometry.box(minx, miny, maxx, maxy)
    polygon = box(*bounds)
    return crs_convert_shapely(polygon, input_crs, out_crs)


## ipyleaflet helpers

In [5]:
from ipyleaflet import ImageOverlay
from PIL import Image
from io import StringIO, BytesIO
from base64 import b64encode

# adapted from https://github.com/jupyter-widgets/ipyleaflet/issues/705
def make_imageoverlay(array, bounds):
    """
    Make ImageOverlay from numpy array.
    
    :arg array: Data in 2D numpy array.
    :arg bounds: Image latitude, longitude bounds, [(lat_min, lon_min), (lat_max, lon_max)]
    """
    image = Image.fromarray(array)
    # store image in memory
    f = BytesIO()
    image.save(f, 'png')
    data = b64encode(f.getvalue())
    data = data.decode('ascii')
    imgurl = 'data:image/png;base64,' + data
    io = ImageOverlay(url=imgurl, bounds=bounds)
    return io

# Validation Image Helpers

In [ ]:
# how the validation dataframe has been created:
import csv
from pathlib import Path
import geopandas as gpd
import json

# columns:
# image_name, is_crossing, crossing_location, invalid_image
fieldnames = [
    "image_name",
    "is_crossing",
    "crossing_location",  # json string
    "invalid_image",
]

def get_validated_images():
    column_conversion = {
        "image_name": str,
        "is_crossing": bool,
        "crossing_location": lambda x: json.loads(x) if x else None,
        "invalid_image": bool,
    }
    if not CLEANED_IMAGES_CSV.exists():
        return []
    with open(CLEANED_IMAGES_CSV, newline='') as csvfile:
        images = csv.DictReader(csvfile)
        results = []
        for row in images:
            row_result = {
                column_name : column_conversion[column_name](row[column_name]) for column_name in row
            }
            results.append(row_result)
    return results

def add_validated_image(*, image_path: Path, is_crossing: bool, crossing_location: str,  invalid_image: bool):
    insert_dict = {
        "image_name": image_path.name,
        "is_crossing": is_crossing,
        "crossing_location": json.dumps(crossing_location),
        "invalid_image": invalid_image,
    }
    if not CLEANED_IMAGES_CSV.exists():
        with open(CLEANED_IMAGES_CSV, newline='', mode='a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

    with open(CLEANED_IMAGES_CSV, newline='', mode='a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(insert_dict)
    already_validate_images = get_validated_images()


## image helpers

In [1]:
import numpy as np
from pathlib import Path
from PIL import Image
import rasterio
import rasterio.mask
from shapely.geometry import box
from shapely.geometry import mapping


def cut_image(src_file, *, filename_polygon_dict: dict, destination_folder: Path, src_crs=CRS_2056, bounding_crs=CRS_4326):
    with cd(Path(src_file).parent):
        with rasterio.open(src_file) as src:
            for fname, polygon in filename_polygon_dict.items():
                polygon = crs_convert_shapely(polygon, in_crs=bounding_crs, out_crs=src_crs)
                try:
                    out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True)
                except Exception as e:
                    print("out of bounds")
                    print(e)
                    continue
                out_meta = src.meta
                out_meta.update({
                    "driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                })
                image_out_path = Path(destination_folder) / f"{fname}.tif"
                with rasterio.open(image_out_path.absolute(), "w", **out_meta) as dest:
                    dest.write(out_image)
    return out_image, out_transform


def cut_image_using_warp(src_file, bounding_boxes: list, destination_folder, src_crs=CRS_2056, dest_crs=CRS_4326):
    with cd(Path(src_file).parent):
        with rasterio.open(src_file) as src:
            with rasterio.vrt.WarpedVRT(src, src_crs=src_crs, crs=dest_crs) as vrt:
                for index, bounding_box in enumerate(bounding_boxes):
                    polygon = box(bounding_box)
                    out_image, out_transform = rasterio.mask.mask(vrt, [polygon.geometry], crop=True)
                    out_meta = src.meta
                    out_meta.update({
                        "driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform,
                    })
                    image_out_path = Path(destination_folder) / f"{str(index+1).zfill(10)}.tif"
                    with rasterio.open(image_out_path.absolute(), "w", **out_meta) as dest:
                        dest.write(out_image)
    return out_image, out_transform



def convert_banded_image(
    image_path,
    out_directory=None,
    bands=[3,1,2],
    convert_to="RGB",
    suffix=".png",
    reduction_factor=6000,
):
    p = Path(image_path)
    if not out_directory:
        out_directory = p.parent

    with rasterio.open(image_path, 'r') as src:
        im_arr = src.read(bands)

    im_reduced = reduce_depth(im_arr, 0, reduction_factor)
    im_reduced = reshape_as_image(im_reduced)
    im = Image.fromarray(im_reduced)
    im = im.convert(convert_to)
    file_out = out_directory / f"{p.stem}{suffix}"
    im.save(file_out.with_suffix(suffix).absolute())


def reduce_depth(image, display_min, display_max, *, target_size=256, as_type=np.uint8):
    image -= display_min
    image = np.floor_divide(image, (display_min - display_max + 1) / target_size)
    image = image.astype(as_type)
    return image



## Buffer Helpers

In [ ]:
from geopandas import GeoSeries
from shapely.geometry import Point, Polygon, shape



def shapely_to_geo_series_epsg_3857(*, shape: shape, input_crs=CRS_4326):
    gs = GeoSeries([shape], crs=input_crs)
    # convert so we can use meters
    return gs.to_crs(epsg=3857)


def shapely_to_geo_series_crs_conversion(shape: shape, *, input_crs, output_crs):
    gs = GeoSeries([shape], crs=input_crs)
    series = gs.to_crs(output_crs)
    converted_shape = series[0]
    return converted_shape


def circle_with_radius(
    *,
    point: Point,
    radius_in_meters: int,
    input_crs=CRS_4326,
    output_crs=CRS_4326,
):
    return buffer_with_radius(
        shape=point,
        radius_in_meters=radius_in_meters,
        input_crs=input_crs,
        output_crs=output_crs,
    )


def buffer_with_radius(
    *,
    shape: shape,
    radius_in_meters: int,
    input_crs=CRS_4326,
    output_crs=CRS_4326,
):
    if radius_in_meters <= 0:
        return shape
    gs = shapely_to_geo_series_epsg_3857(shape=shape, input_crs=input_crs)
    radius_to_buffer = radius_in_meters
    shape = gs.buffer(radius_to_buffer)
    area = shape.to_crs(output_crs)
    return area.geometry[0]


def buffered_shape(
    *,
    shape: shape,
    radius_in_meters: int,
    input_crs=CRS_4326,
    output_crs=CRS_4326,
) -> Polygon:
    kwargs = dict(
        shape=shape,
        radius_in_meters=radius_in_meters,
        input_crs=input_crs,
        output_crs=output_crs,
    )
    buffered = buffer_with_radius(**kwargs)
    return Polygon.from_bounds(*buffered.bounds)



# One-off stuff

## find a decent reduction value

In [1]:
import geopandas as gpd
import rasterio as rio
from shapely.geometry import box
from rasterio.plot import reshape_as_raster, reshape_as_image

def show_example_image_with_factor(image, factor):
    EXAMPLE_IMAGE = IMAGES_TIF_FOLDER_CROSSINGS / "8.474699974060059_47.40547180175781_is_crossing_1.tif"
    EXAMPLE_IMAGE.exists()

    with rio.open(EXAMPLE_IMAGE, 'r') as src:
        bounds = shapely_to_geo_series_crs_conversion(shape=box(*src.bounds), input_crs=CRS_2056, output_crs=CRS_4326)
        wgs84_center = [bounds.centroid.y,bounds.centroid.x]
        image_data = src.read([3,1,2])
        reduced = reduce_depth(image_data, display_min=0, display_max=reduction_factor)
    rio.plot.show(reduced)

In [ ]:
# uncomment to run

# reduction_factor = 6000
# example_image = next(IMAGES_TIF_FOLDER_CROSSINGS.glob('*.tif'))
# show_example_image_with_factor(example_image, reduction_factor)

In [ ]:
# relicts: finding image border
# image_bounds = bounds.bounds
# image_bounds = [[image_bounds[3], image_bounds[2]],[image_bounds[1], image_bounds[0]]]
# print(wgs84_center)
# print(image_bounds)